In [1]:
import numpy as np
import pandas as pd

**对于获得的数据，会存在内容缺失，表格轴名称错误，数据类型需要改变等问题，所以在数据分析前，我们需要将数据做一些基本的操作：清理不必要的的数据，将缺失数据填充。**

# 缺失数据处理

## 滤除缺失数据

**判断数据是否是拥有空值，使用pandas的isnull()和notnull()**

In [2]:
data = pd.Series(['a','d','f','g',np.nan,'j',np.nan])

In [3]:
data

0      a
1      d
2      f
3      g
4    NaN
5      j
6    NaN
dtype: object

In [4]:
pd.isnull(data)

0    False
1    False
2    False
3    False
4     True
5    False
6     True
dtype: bool

In [5]:
pd.notnull(data)

0     True
1     True
2     True
3     True
4    False
5     True
6    False
dtype: bool

**如果想把一位数变为nan，你可以使用None**

In [6]:
data[2] = None

In [7]:
data

0       a
1       d
2    None
3       g
4     NaN
5       j
6     NaN
dtype: object

In [8]:
pd.isnull(data)

0    False
1    False
2     True
3    False
4     True
5    False
6     True
dtype: bool

**滤除空的值，我们一般使用dropna()**

In [9]:
data.dropna()

0    a
1    d
3    g
5    j
dtype: object

**可以看出来，只要是行中有nan的数据都被以行的形式删除了。**

**对于Series的数据，直接使用dropna()是方便的。如果是DataFrame类型，我们就需要更加复杂的方法**


In [10]:
frame = pd.DataFrame([[1,2,np.nan],[9,1,np.nan],[89,np.nan,np.nan],[1,2,np.nan]])  #这是一个二维数组

In [11]:
frame

,0,1,2
0,1,2.0,NaN
1,9,1.0,NaN
2,89,NaN,NaN
3,1,2.0,NaN



**我们对DataFrame类型使用dropna()函数**

In [12]:
frame.dropna()

,0,1,2


**可以看到，上面单独使用dropna(),只要里面有nan的就会被整行删除**

**如果想让删除的条件没那么苛刻，比如条件改为将全行为nan的数删除，那可以增加参数how='all'**

In [13]:
frame.dropna(how = 'all')

,0,1,2
0,1,2.0,NaN
1,9,1.0,NaN
2,89,NaN,NaN
3,1,2.0,NaN


**如果我将考虑的是列，输入axis=1，改变所考虑的行就好了**

In [14]:
frame.dropna(axis=1)

,0
0,1
1,9
2,89
3,1


**同理，如果想将全列为nan的数据清除，也可以使用how='all'**

In [15]:
frame.dropna(axis=1,how='all')

,0,1
0,1,2.0
1,9,1.0
2,89,NaN
3,1,2.0


**我想更好的空值清除数据的条件，比如我要决定这个行数据，要将保留n和非nan数据的行留下，这个时候可以使用dropna()中的thresh=**

In [16]:
 frame = pd.DataFrame(np.ceil(np.random.uniform(1,100,(4,7))))

In [17]:
frame

,0,1,2,3,4,5,6
0,88.0,91.0,14.0,93.0,82.0,95.0,92.0
1,68.0,63.0,5.0,58.0,13.0,93.0,8.0
2,16.0,95.0,57.0,22.0,70.0,14.0,70.0
3,67.0,73.0,74.0,61.0,40.0,37.0,73.0


In [18]:
frame.iloc[0:2,1]= np.nan
frame.iloc[0:3,5]= np.nan

In [19]:
frame

,0,1,2,3,4,5,6
0,88.0,NaN,14.0,93.0,82.0,NaN,92.0
1,68.0,NaN,5.0,58.0,13.0,NaN,8.0
2,16.0,95.0,57.0,22.0,70.0,NaN,70.0
3,67.0,73.0,74.0,61.0,40.0,37.0,73.0


In [20]:
frame.dropna()

,0,1,2,3,4,5,6
3,67.0,73.0,74.0,61.0,40.0,37.0,73.0


In [21]:
frame.dropna(thresh=6)    

,0,1,2,3,4,5,6
2,16.0,95.0,57.0,22.0,70.0,NaN,70.0
3,67.0,73.0,74.0,61.0,40.0,37.0,73.0


## 数据填充

**数据空缺，我们将其这样的数据过滤是一种方法，但是如果我们将这些数据填充起来也是一种选择**

**填充的方法是：fillna()**

**填充的思路有：1.填充一个专门的数 2.填充一这类类型的数（比如平均数，众数等）3.填充前面有值的那个数，或者填充后面的那个数**

In [22]:
#1.填充一个专门的数
frame = pd.DataFrame([[1,2,15],[9,1,np.nan],[89,np.nan,np.nan],[1,2,np.nan]])  #这是一个二维数组

In [23]:
frame 

,0,1,2
0,1,2.0,15.0
1,9,1.0,NaN
2,89,NaN,NaN
3,1,2.0,NaN


In [24]:
frame.fillna(1000)

,0,1,2
0,1,2.0,15.0
1,9,1.0,1000.0
2,89,1000.0,1000.0
3,1,2.0,1000.0


In [25]:
frame[1].mean()

1.6666666666666667

In [26]:
#2.填充一这类类型的数（比如平均数，众数等）
frame[1].fillna(frame[1].mean(),inplace=True)  #第一列的数据被第一列的元素的平均值填充

In [27]:
frame

,0,1,2
0,1,2.000000,15.0
1,9,1.000000,NaN
2,89,1.666667,NaN
3,1,2.000000,NaN


In [28]:
frame[2].fillna(method='ffill',inplace=True)

In [29]:
frame

,0,1,2
0,1,2.000000,15.0
1,9,1.000000,15.0
2,89,1.666667,15.0
3,1,2.000000,15.0


# 数据转换

## 移除重复数据

**使用最常用的移除重复数据的方法是drop_duplicates(),查明是不是重复数据的方法是duplicates()**

In [30]:
data = pd.DataFrame({'名字':['小明','小白','小伙子']*3+['小张'],'年龄':['19','20','89']*3+['29']})

In [31]:
data


,名字,年龄
0,小明,19
1,小白,20
2,小伙子,89
3,小明,19
4,小白,20
5,小伙子,89
6,小明,19
7,小白,20
8,小伙子,89
9,小张,29


In [32]:
data.duplicated()

0    False
1    False
2    False
3     True
4     True
5     True
6     True
7     True
8     True
9    False
dtype: bool

In [33]:
data.drop_duplicates()

,名字,年龄
0,小明,19
1,小白,20
2,小伙子,89
9,小张,29


In [34]:
#对于特定列的排除
data.drop_duplicates(['名字'])

,名字,年龄
0,小明,19
1,小白,20
2,小伙子,89
9,小张,29


In [35]:
#对于特定列的排除
data.drop_duplicates(['名字'])

,名字,年龄
0,小明,19
1,小白,20
2,小伙子,89
9,小张,29


**我们检查重复的数据，都是从前往后查，如果我们从后往前查，就需要keep=参数**

In [36]:
#对于特定列的排除
data.drop_duplicates(['名字'],keep='last')

,名字,年龄
6,小明,19
7,小白,20
8,小伙子,89
9,小张,29


## 利用函数或映射进行数据转换

**在Series类中，可以传入map类型，可以做map匹配**

**首先注意DataFrame要变成Series只需拿到某一列就好了**

In [37]:
Naruto = pd.DataFrame({'角色':['Mingren','Zuozhu','Dashewan','Kakaxi'],'战斗力':['99','99','100','89']})

In [38]:
Naruto

,角色,战斗力
0,Mingren,99
1,Zuozhu,99
2,Dashewan,100
3,Kakaxi,89


In [39]:
#获得一个Series的类型
line = Naruto['角色']

In [40]:
#产生一个map数据
mapline = {'mingren':'88','zuozhu':'100','dashewan':'89','kakaxi':'80'}
mapline

{'mingren': '88', 'zuozhu': '100', 'dashewan': '89', 'kakaxi': '80'}

**我们注意到，新产生的map的key是小写的，但是产生的Series类型是大写的，所以我们要做一个转换，然后再做映射**

In [41]:
line = Naruto['角色'].str.lower()
Naruto['成长值'] = line.map(mapline)

In [42]:
Naruto

,角色,战斗力,成长值
0,Mingren,99,88
1,Zuozhu,99,100
2,Dashewan,100,89
3,Kakaxi,89,80


## 替换值

**我们之前可以一些数字的填充，但是fillna()一般用于整行和整列的填充，所以对于特定数据的填充，我们可以使用数值替换的方法replace()**

In [43]:
data = pd.Series([1,2,999,-999,9,6,7])

In [44]:
data

0      1
1      2
2    999
3   -999
4      9
5      6
6      7
dtype: int64

In [45]:
data.replace([999,-999],[100,200])

0      1
1      2
2    100
3    200
4      9
5      6
6      7
dtype: int64

In [46]:
data.replace({999:100,-999:200}) #可以传入字典

0      1
1      2
2    100
3    200
4      9
5      6
6      7
dtype: int64

## 重命名轴索引

**我们在DataFrame中，一些轴的数据是要要根据你的用途，不断地修改**

In [47]:
Naruto = pd.DataFrame({'actor':['Mingren','Zuozhu','Dashewan','Kakaxi'],'pow':['99','99','100','89']})

In [48]:
Naruto

,actor,pow
0,Mingren,99
1,Zuozhu,99
2,Dashewan,100
3,Kakaxi,89


**我们列名大写**

In [49]:
Naruto.columns

Index(['actor', 'pow'], dtype='object')

In [50]:
transfer = lambda x : x[:].upper()

In [51]:
Naruto.columns.map(transfer)

Index(['ACTOR', 'POW'], dtype='object')

**对于一个轴，如果你想改变的名称，比较好用的方法是rename(),通过字典的形式传入**

In [52]:
Naruto.rename(index={0:'火影村',1:'火影村',2:'音影村',3:'火影村'},columns={'actor':'角色','pow':'力量'})

,角色,力量
火影村,Mingren,99
火影村,Zuozhu,99
音影村,Dashewan,100
火影村,Kakaxi,89


## 离散化和面元划分

**所谓的面元是指，自己对一组元素自定义划分方法，原数据被这种规定划分的过程是面元划分**

In [53]:
args = np.ceil(np.random.uniform(1,100,8))

In [54]:
args

array([20., 86., 79., 66., 95., 79., 19., 99.])

In [55]:
#编写划分的区间
bin = [10,30,50,70,100]  #注意区间的排列方式

**面元划分的方法pd.cut()**

In [56]:
cuts = pd.cut(args,bin)

In [57]:
cuts #返回的值是自己所在的区间

[(10, 30], (70, 100], (70, 100], (50, 70], (70, 100], (70, 100], (10, 30], (70, 100]]
Categories (4, interval[int64]): [(10, 30] < (30, 50] < (50, 70] < (70, 100]]

**对于原数据，我们想知道在哪个区间的位置表示，使用pandas的codes的标签**

In [58]:
cuts.codes  #这种情况也可以当做将一列区间的数据离散化

array([0, 3, 3, 2, 3, 3, 0, 3], dtype=int8)

**下面的需求变得自然而然，在我将我的数据离散化之后，在数据很大的情况下，我需要统计我的数据在这些区间的个数**

**这个时候使用的是value_counts()**

In [59]:
pd.value_counts(cuts)

(70, 100]    5
(10, 30]     2
(50, 70]     1
(30, 50]     0
dtype: int64

**面元之后，我的们得到了离散的数据，这里的数据都是左开右闭，right=**

In [60]:
cuts = pd.cut(args,bin,right=False)
cuts

[[10, 30), [70, 100), [70, 100), [50, 70), [70, 100), [70, 100), [10, 30), [70, 100)]
Categories (4, interval[int64]): [[10, 30) < [30, 50) < [50, 70) < [70, 100)]

**如上面提到，我们分为的离散型数据如果不用区间或者区间所代表的的位置表示，而是用这个区间所代表的名字表示(string类型的数据)，将会更加直白**

In [61]:
names = ['少','中等','稍多','多']
cuts  = pd.cut(args,bin,labels=names)

In [62]:
cuts

[少, 多, 多, 稍多, 多, 多, 少, 多]
Categories (4, object): [少 < 中等 < 稍多 < 多]

**在数据的使用圆面划分时，除了使用区间值的这种方法，我们还可以用样本分位数做划分**

**这个时候使用的方法是qcut()**

In [63]:
qcuts = pd.qcut(args,[0.2,0.5,0.9,1],precision=2)

In [64]:
qcuts

[NaN, (79.0, 96.2], (38.39, 79.0], (38.39, 79.0], (79.0, 96.2], (38.39, 79.0], NaN, (96.2, 99.0]]
Categories (3, interval[float64]): [(38.39, 79.0] < (79.0, 96.2] < (96.2, 99.0]]

**precision=表示的，小数的精度为两位小数**

**像下面这种情况**

In [65]:
qcuts = pd.qcut(args,5,precision=2)

In [66]:
qcuts

[(18.99, 38.4], (80.4, 91.4], (76.4, 80.4], (38.4, 76.4], (91.4, 99.0], (76.4, 80.4], (18.99, 38.4], (91.4, 99.0]]
Categories (5, interval[float64]): [(18.99, 38.4] < (38.4, 76.4] < (76.4, 80.4] < (80.4, 91.4] < (91.4, 99.0]]

In [67]:
pd.value_counts(qcuts)

(91.4, 99.0]     2
(76.4, 80.4]     2
(18.99, 38.4]    2
(80.4, 91.4]     1
(38.4, 76.4]     1
dtype: int64

**输入数字5，是分为5段面元，离散的时候，保证每个面元的数量基本相等**

## 检测和过滤异常值

**我们上面有一个部分展示的是过滤缺失值，使用的是dropna()**

# *一件事情的做成是：强度够，专注时间够*

**而此时我们讨论的是，如果数据是完整的，但是我们想要的数据是具有某些特性的，那么我们需要有过滤不合规范数据的能力**

**有一组DataFrame数据，我们使用describe()方法对这组数据做一个评判**

In [68]:
frame = pd.DataFrame(np.ceil(np.random.uniform(1,1000,(100,6))))

In [69]:
frame

,0,1,2,3,4,5
0,96.0,389.0,957.0,876.0,39.0,91.0
1,727.0,560.0,916.0,9.0,32.0,298.0
2,364.0,10.0,389.0,479.0,204.0,4.0
3,423.0,156.0,593.0,850.0,215.0,726.0
4,897.0,140.0,449.0,97.0,805.0,34.0
5,900.0,838.0,737.0,596.0,165.0,581.0
6,735.0,153.0,338.0,126.0,559.0,214.0
7,504.0,53.0,109.0,819.0,378.0,540.0
8,654.0,394.0,733.0,980.0,602.0,244.0
9,377.0,774.0,651.0,209.0,97.0,717.0


In [70]:
frame.describe() #记录了每一列的数据量。均值，最大，最小，分位数，标准差(std)

,0,1,2,3,4,5
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,520.660000,502.480000,506.370000,503.760000,494.890000,422.790000
std,280.413735,294.772553,262.940315,288.326629,278.567617,274.850935
min,3.000000,9.000000,7.000000,3.000000,8.000000,4.000000
25%,260.250000,259.000000,312.250000,285.750000,304.000000,203.500000
50%,565.500000,482.500000,491.500000,533.000000,460.500000,386.000000
75%,740.000000,776.000000,724.000000,770.500000,715.500000,640.250000
max,980.000000,978.000000,982.000000,999.000000,986.000000,964.000000


**PS:当一个DataFrame型数据，取其一行或者一列的时候，这时候数据就变成了Series类型**

In [71]:
data = frame[4]

In [72]:
data

0      39.0
1      32.0
2     204.0
3     215.0
4     805.0
5     165.0
6     559.0
7     378.0
8     602.0
9      97.0
10    768.0
11    579.0
12     64.0
13    414.0
14    567.0
15    344.0
16    411.0
17    248.0
18    803.0
19    250.0
20    236.0
21    869.0
22    629.0
23    664.0
24    332.0
25    723.0
26    345.0
27    876.0
28    713.0
29     41.0
      ...  
70    536.0
71    921.0
72    361.0
73    384.0
74    796.0
75    383.0
76     30.0
77    393.0
78    366.0
79    397.0
80    886.0
81    585.0
82    101.0
83    397.0
84    446.0
85     12.0
86    786.0
87    958.0
88    610.0
89     32.0
90    123.0
91     82.0
92    978.0
93    986.0
94    965.0
95    902.0
96    666.0
97    581.0
98    155.0
99    582.0
Name: 4, Length: 100, dtype: float64

**我们将Series类的data中，大于500的数字挑选出来**

In [73]:
data[data>500]

4     805.0
6     559.0
8     602.0
10    768.0
11    579.0
14    567.0
18    803.0
21    869.0
22    629.0
23    664.0
25    723.0
27    876.0
28    713.0
31    593.0
33    567.0
34    975.0
38    579.0
40    835.0
41    875.0
43    879.0
44    974.0
45    681.0
46    551.0
47    844.0
48    624.0
49    562.0
57    830.0
59    844.0
60    640.0
62    835.0
64    644.0
65    785.0
66    506.0
70    536.0
71    921.0
74    796.0
80    886.0
81    585.0
86    786.0
87    958.0
88    610.0
92    978.0
93    986.0
94    965.0
95    902.0
96    666.0
97    581.0
99    582.0
Name: 4, dtype: float64

**对于DataFrame类型的数据的操作也是一样的**

In [74]:
#找到frame中数据的大于500的数字
frame[frame>500]

,0,1,2,3,4,5
0,NaN,NaN,957.0,876.0,NaN,NaN
1,727.0,560.0,916.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,593.0,850.0,NaN,726.0
4,897.0,NaN,NaN,NaN,805.0,NaN
5,900.0,838.0,737.0,596.0,NaN,581.0
6,735.0,NaN,NaN,NaN,559.0,NaN
7,504.0,NaN,NaN,819.0,NaN,540.0
8,654.0,NaN,733.0,980.0,602.0,NaN
9,NaN,774.0,651.0,NaN,NaN,717.0


**如果将那些小于500的数字都变成500**

In [75]:
frame[frame<500]=500

In [76]:
frame[frame>=500]

,0,1,2,3,4,5
0,500.0,500.0,957.0,876.0,500.0,500.0
1,727.0,560.0,916.0,500.0,500.0,500.0
2,500.0,500.0,500.0,500.0,500.0,500.0
3,500.0,500.0,593.0,850.0,500.0,726.0
4,897.0,500.0,500.0,500.0,805.0,500.0
5,900.0,838.0,737.0,596.0,500.0,581.0
6,735.0,500.0,500.0,500.0,559.0,500.0
7,504.0,500.0,500.0,819.0,500.0,540.0
8,654.0,500.0,733.0,980.0,602.0,500.0
9,500.0,774.0,651.0,500.0,500.0,717.0


## 排列和随机采样

**当我们有一个Series和DataFrame的数据，数据本身我们不希望它再改变，但我们希望数据的排列顺序和采样的方式发生变化，那么我们下面来讨论这个**

**对于numpy里面的数据，我们可以使用numpy.random.permutation()这个方法**

In [77]:
np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [78]:
test = np.random.permutation(10)
test

array([0, 6, 8, 4, 1, 9, 7, 3, 2, 5])

In [79]:
frame = pd.DataFrame(np.ceil(np.random.uniform(1,1000,(10,6))))
frame

,0,1,2,3,4,5
0,553.0,881.0,753.0,711.0,415.0,610.0
1,91.0,783.0,653.0,583.0,861.0,863.0
2,992.0,404.0,811.0,134.0,464.0,613.0
3,108.0,858.0,623.0,364.0,397.0,855.0
4,142.0,376.0,541.0,105.0,759.0,853.0
5,93.0,413.0,326.0,67.0,226.0,428.0
6,479.0,64.0,321.0,19.0,316.0,955.0
7,736.0,514.0,996.0,217.0,681.0,613.0
8,945.0,200.0,426.0,275.0,978.0,186.0
9,500.0,660.0,663.0,244.0,822.0,871.0


**我们使用take()方法，方法里面传入的是我们想要的顺序**

In [80]:
frame.take(test)

,0,1,2,3,4,5
0,553.0,881.0,753.0,711.0,415.0,610.0
6,479.0,64.0,321.0,19.0,316.0,955.0
8,945.0,200.0,426.0,275.0,978.0,186.0
4,142.0,376.0,541.0,105.0,759.0,853.0
1,91.0,783.0,653.0,583.0,861.0,863.0
9,500.0,660.0,663.0,244.0,822.0,871.0
7,736.0,514.0,996.0,217.0,681.0,613.0
3,108.0,858.0,623.0,364.0,397.0,855.0
2,992.0,404.0,811.0,134.0,464.0,613.0
5,93.0,413.0,326.0,67.0,226.0,428.0


In [81]:
frame.take(test)

,0,1,2,3,4,5
0,553.0,881.0,753.0,711.0,415.0,610.0
6,479.0,64.0,321.0,19.0,316.0,955.0
8,945.0,200.0,426.0,275.0,978.0,186.0
4,142.0,376.0,541.0,105.0,759.0,853.0
1,91.0,783.0,653.0,583.0,861.0,863.0
9,500.0,660.0,663.0,244.0,822.0,871.0
7,736.0,514.0,996.0,217.0,681.0,613.0
3,108.0,858.0,623.0,364.0,397.0,855.0
2,992.0,404.0,811.0,134.0,464.0,613.0
5,93.0,413.0,326.0,67.0,226.0,428.0


**对于Series和DataFrame类型数据，使用sample()方法，传入n=，输出随机的等量的数据**

In [82]:
frame.sample(n=5)

,0,1,2,3,4,5
8,945.0,200.0,426.0,275.0,978.0,186.0
7,736.0,514.0,996.0,217.0,681.0,613.0
4,142.0,376.0,541.0,105.0,759.0,853.0
3,108.0,858.0,623.0,364.0,397.0,855.0
6,479.0,64.0,321.0,19.0,316.0,955.0


## 计算指标/哑变量

**什么是哑变量：https://baike.baidu.com/item/%E8%99%9A%E6%8B%9F%E5%8F%98%E9%87%8F/8262721?fromtitle=%E5%93%91%E5%8F%98%E9%87%8F&fromid=9621990**

**先拿一个数据做例子看一下，在pandas中有一个get_dummies()方法可以完成**

In [83]:
frame = pd.DataFrame({'key':['a','b','c','a','b','r','s'],'detail':[1,2,3,4,5,6,7]})

In [84]:
frame

,key,detail
0,a,1
1,b,2
2,c,3
3,a,4
4,b,5
5,r,6
6,s,7


In [85]:
pd.get_dummies(frame['key'])

,a,b,c,r,s
0,1,0,0,0,0
1,0,1,0,0,0
2,0,0,1,0,0
3,1,0,0,0,0
4,0,1,0,0,0
5,0,0,0,1,0
6,0,0,0,0,1


**有时候你的列名需要加前缀，使用的是prefix=参数，为了和另一个DataFrame类型的数据合并(合并的方法使用的是jion())**

In [86]:
dummies = pd.get_dummies(frame['key'],prefix='key')

In [87]:
double = frame[['detail']].join(dummies)   #[[ ]]这样在DataFrame类型的数据取数据时，可以保证取出来的数据是DataFrame类型

In [88]:
double

,detail,key_a,key_b,key_c,key_r,key_s
0,1,1,0,0,0,0
1,2,0,1,0,0,0
2,3,0,0,1,0,0
3,4,1,0,0,0,0
4,5,0,1,0,0,0
5,6,0,0,0,1,0
6,7,0,0,0,0,1


**对一个统计有用的方法是，使用get_dummies和诸如cut这样的离散化函数会很有帮助**

In [89]:
np.random.seed(123)    

In [90]:
data = np.random.rand(10)

In [91]:
data

array([0.69646919, 0.28613933, 0.22685145, 0.55131477, 0.71946897,
       0.42310646, 0.9807642 , 0.68482974, 0.4809319 , 0.39211752])

In [92]:
test = pd.cut(data,4)
dummies = pd.get_dummies(test)

In [93]:
dummies

   (0.226, 0.415]  (0.415, 0.604]  (0.604, 0.792]  (0.792, 0.981]
0               0               0               1               0
1               1               0               0               0
2               1               0               0               0
3               0               1               0               0
4               0               0               1               0
5               0               1               0               0
6               0               0               0               1
7               0               0               1               0
8               0               1               0               0
9               1               0               0               0

# 字符串操作

## 字符串对象方法

**Python是一个对于字符串操作很友好的编程语言，pandas对这个操作做了加强**

**一般的对象操作是使用split()方法分隔字符**

In [94]:
string = 'lala,haha, oyou'

In [95]:
a = string.split(',')
a

['lala', 'haha', ' oyou']

**strip()可以去除空格,strip()方法用在string上面**

In [96]:
values = [i.strip() for i in a]

In [97]:
values

['lala', 'haha', 'oyou']

In [98]:
# Python中的lsit可以做一个扩展
name,attitude,sayhi  = values

**利用加法原则将数据连接起来**

In [99]:
name+'::'+attitude+"::"+sayhi

'lala::haha::oyou'

**对于string数据的查找，我们使用的是in这个判断方法，对于这元素所在的位置，我们使用的是index好find**

**关于index和find的用法和区别：https://blog.csdn.net/u013203733/article/details/72763773**

In [104]:
'lala'in values

True

In [105]:
string.index(',')

4

In [106]:
string.find(',')

4

In [109]:
string.index('9') #如果找不到就会抛出异常，如果find()找不到，那么返回-1

ValueError: substring not found

**如果是统计string里面的数据多少，使用的是count()**

In [110]:
string.count(',')

2

**在string内部的某些元素需要替换，使用replace()**

In [111]:
string.replace(',','::')

'lala::haha:: oyou'

In [113]:
string.replace(',','')  #这样就删除了

'lalahaha oyou'

## Pandas字符串空缺的解决方法

**对于数据缺失，更麻烦的是字符串的缺失，如果对字符串做一些操作，遇到空缺的地方就会报错**

**Series类型的通过str属性可以访问字符串数据，可以忽略NA值**                                                         
我是这样理解的：nan是float的形式，str是string，所以可以忽略

In [120]:
data = {'lala':'fuxiao@com','anqi':'xinhua@com','bama':np.nan}

In [121]:
influence = pd.Series(data)

In [127]:
influence

lala    fuxiao@com
anqi    xinhua@com
bama           NaN
dtype: object

In [129]:
influence.contains('com')

AttributeError: 'Series' object has no attribute 'contains'

In [125]:
influence.str.contains('com')

lala    True
anqi    True
bama     NaN
dtype: object

## 正则表达式

**Python是一个支持正则的语言（很多语言都支持），调用Python的re库，可以使用**

**对于正则，一般分为三个主要的作用：匹配，分隔和替换**

**举一个例子**

In [130]:
import re

In [131]:
test = 'I am chenandong in\t university \t  '

In [132]:
re.split('\s+',test)

['I', 'am', 'chenandong', 'in', 'university', '']

**也可以先对这个做编译，将编译的结果存储，直接调用**

In [134]:
regx = re.compile('\s+')

In [135]:
regx.split(test)

['I', 'am', 'chenandong', 'in', 'university', '']

**如果你是多次想做这同一件是，你可以先编译，这样会提高速度，因为CPU可以节省大量的时间**

**你想要的东西是你被正则所选取的，那你就是使用findall()**

In [136]:
regx.findall(test)

[' ', ' ', ' ', '\t ', ' \t  ']

**对于正则，这个东西需要你的认真学习，但是这里不做展开，对于基础的入门，你可是看看：https://www.liaoxuefeng.com/wiki/1016959663602400/1017639890281664**